<a href="https://colab.research.google.com/github/mahakgarg333/Avataar/blob/main/Avataar.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Task 1: Segment Object Code

In [1]:
pip install torch torchvision segment-anything diffusers transformers

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.7/2.7 MB 21.9 MB/s eta 0:00:00


In [2]:
!wget https://dl.fbaipublicfiles.com/segment_anything/sam_vit_h_4b8939.pth


--2024-10-04 09:24:22--  https://dl.fbaipublicfiles.com/segment_anything/sam_vit_h_4b8939.pth
Resolving dl.fbaipublicfiles.com (dl.fbaipublicfiles.com)... 18.238.136.37, 18.238.136.120, 18.238.136.52, ...
Connecting to dl.fbaipublicfiles.com (dl.fbaipublicfiles.com)|18.238.136.37|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2564550879 (2.4G) [binary/octet-stream]
Saving to: ‘sam_vit_h_4b8939.pth’

sam_vit_h_4b8939.pt 100%[===================>]   2.39G   107MB/s    in 25s     

2024-10-04 09:24:47 (98.3 MB/s) - ‘sam_vit_h_4b8939.pth’ saved [2564550879/2564550879]



In [6]:
def load_sam_model():
    sam_checkpoint = "/content/sam_vit_h_4b8939.pth"  # Correct path to the downloaded file
    model_type = "vit_h"
    sam = sam_model_registry[model_type](checkpoint=sam_checkpoint)
    return sam


In [4]:
!ls /content


chair.jpg  sample_data	sam_vit_h_4b8939.pth


In [5]:
import torch
from segment_anything import sam_model_registry, SamPredictor
import cv2
import numpy as np

# Load the SAM model
def load_sam_model():
    sam_checkpoint = "/content/sam_vit_h_4b8939.pth"  # Correct path after downloading the model
    model_type = "vit_h"
    sam = sam_model_registry[model_type](checkpoint=sam_checkpoint)
    return sam

def segment_object(image_path, object_class, output_path):
    # Load SAM model
    sam = load_sam_model()
    predictor = SamPredictor(sam)

    # Read the image
    image = cv2.imread(image_path)
    predictor.set_image(image)

    # Dummy object prompt for bounding box (Replace with actual detection model for the object class)
    # This example uses a static bounding box; in a real scenario, you would replace it with automatic detection
    box = np.array([50, 50, 300, 300])  # Example bounding box for testing

    # Predict mask for the detected object
    masks, _, _ = predictor.predict(box=box)

    # Apply the mask on the image
    mask_image = np.zeros_like(image)
    mask_image[masks[0]] = [0, 0, 255]  # Red color mask for the object

    # Blend original image and mask
    output_image = cv2.addWeighted(image, 0.7, mask_image, 0.3, 0)

    # Save the output
    cv2.imwrite(output_path, output_image)
    print(f"Segmentation result saved to {output_path}")

# Example usage
image_path = '/content/chair.jpg'  # Path to your input image
object_class = 'chair'  # The class of the object you want to segment
output_path = '/content/segmented.png'  # Output file path

segment_object(image_path, object_class, output_path)


/usr/local/lib/python3.10/dist-packages/segment_anything/build_sam.py:105: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  state_dict = torch.load(f)


Segmentation result saved to /content/segmented.png


Task 2: Pose Editing Code

In [12]:
import torch
from segment_anything import sam_model_registry, SamPredictor
import cv2
import numpy as np

# Load the SAM model
def load_sam_model():
    sam_checkpoint = "/content/sam_vit_h_4b8939.pth"  # Correct path after downloading the model
    model_type = "vit_h"
    sam = sam_model_registry[model_type](checkpoint=sam_checkpoint)
    return sam

def segment_object(image_path, object_class, output_path, azimuth_angle=0, polar_angle=0):
    # Load SAM model
    sam = load_sam_model()
    predictor = SamPredictor(sam)

    # Read the image
    image = cv2.imread(image_path)
    predictor.set_image(image)

    # Dummy object prompt for bounding box (Replace with actual detection model for the object class)
    # This example uses a static bounding box; in a real scenario, you would replace it with automatic detection
    box = np.array([50, 50, 300, 300])  # Example bounding box for testing

    # Predict mask for the detected object
    masks, _, _ = predictor.predict(box=box)

    # Apply the mask on the image
    mask_image = np.zeros_like(image)
    mask_image[masks[0]] = [0, 0, 255]  # Red color mask for the object

    # Get the object mask
    object_mask = np.zeros_like(image)
    object_mask[masks[0]] = image[masks[0]]

    # Rotate the object
    (h, w) = object_mask.shape[:2]
    center = (w // 2, h // 2)
    M = cv2.getRotationMatrix2D(center, azimuth_angle, 1.0)
    rotated_object = cv2.warpAffine(object_mask, M, (w, h))

    # Create a mask for the rotated object
    rotated_mask = np.zeros_like(rotated_object)
    rotated_mask[rotated_object != 0] = 255

    # Remove the rotated object from the original image
    image_without_object = np.copy(image)
    image_without_object[masks[0]] = 0

    # Add the rotated object to the original image
    output_image = np.copy(image_without_object)
    output_image[rotated_mask != 0] = rotated_object[rotated_mask != 0]

    # Save the output
    cv2.imwrite(output_path, output_image)
    print(f"Segmentation result saved to {output_path}")

# Example usage
image_path = '/content/chair.jpg'  # Path to your input image
object_class = 'chair'  # The class of the object you want to segment
output_path = '/content/rotated.png'  # Output file path
azimuth_angle = 72  # Azimuth angle of rotation
polar_angle = 0  # Polar angle of rotation (not used in this example)

segment_object(image_path, object_class, output_path, azimuth_angle, polar_angle)

Segmentation result saved to /content/rotated.png


In [13]:
import torch
from segment_anything import sam_model_registry, SamPredictor
import cv2
import numpy as np

# Load the SAM model
def load_sam_model():
    sam_checkpoint = "/content/sam_vit_h_4b8939.pth"  # Correct path after downloading the model
    model_type = "vit_h"
    sam = sam_model_registry[model_type](checkpoint=sam_checkpoint)
    return sam

def segment_object(image_path, object_class, output_path, azimuth_angle=0, polar_angle=0):
    # Load SAM model
    sam = load_sam_model()
    predictor = SamPredictor(sam)

    # Read the image
    image = cv2.imread(image_path)
    predictor.set_image(image)

    # Dummy object prompt for bounding box (Replace with actual detection model for the object class)
    # This example uses a static bounding box; in a real scenario, you would replace it with automatic detection
    box = np.array([50, 50, 300, 300])  # Example bounding box for testing

    # Predict mask for the detected object
    masks, _, _ = predictor.predict(box=box)

    # Apply the mask on the image
    mask_image = np.zeros_like(image)
    mask_image[masks[0]] = [0, 0, 255]  # Red color mask for the object

    # Get the object mask
    object_mask = np.zeros_like(image)
    object_mask[masks[0]] = image[masks[0]]

    # Get the bounding box of the object
    x, y, w, h = box

    # Crop the object from the original image
    object_image = image[y:y+h, x:x+w]

    # Rotate the object
    (h, w) = object_image.shape[:2]
    center = (w // 2, h // 2)
    M = cv2.getRotationMatrix2D(center, azimuth_angle, 1.0)
    rotated_object = cv2.warpAffine(object_image, M, (w, h))

    # Create a new image with the same size as the original image
    new_image = np.copy(image)

    # Remove the original object from the new image
    new_image[y:y+h, x:x+w] = 0

    # Get the bounding box of the rotated object
    (h, w) = rotated_object.shape[:2]
    x = (image.shape[1] - w) // 2
    y = (image.shape[0] - h) // 2

    # Add the rotated object to the new image
    new_image[y:y+h, x:x+w] = rotated_object

    # Save the output
    cv2.imwrite(output_path, new_image)
    print(f"Segmentation result saved to {output_path}")

# Example usage
image_path = '/content/chair.jpg'  # Path to your input image
object_class = 'chair'  # The class of the object you want to segment
output_path = '/content/rotated.png'  # Output file path
azimuth_angle = 72  # Azimuth angle of rotation
polar_angle = 0  # Polar angle of rotation (not used in this example)

segment_object(image_path, object_class, output_path, azimuth_angle, polar_angle)

Segmentation result saved to /content/rotated.png
